# Document Goal
The purpose of this notebook is:
- to understand and choose a Markdown text splitter.  Is the splitter splitting in a way that makes sense?
- to evaluate the contents of the text within the returned text splits.  Should the text be cleaned, are there nodes that are too large or too small?

In [ ]:
# This notebook is in the eval folder.  Change to the root folder.
%cd ..
%pwd  # To verify the current working directory

In [2]:
# This is a cool library for printing data in a way that is easy to read.
from rich import print


# Langchain's MarkdownTextSplitter
I'm focusing on Langchain's splitters for now. I tried LlamaIndex's Markdown splitter but did not like the aggressiveness of the splitting.  First I'll try Langchain's `MarkdownTextSplitter`.

The `MarkdownTextSplitter` is a `RecursiveCharacterTextSplitter` that has set the separators to include the Markdown headers. Here is how the splitter will split the text (see markdown.py):

```
 elif language == Language.MARKDOWN:
            return [
                # First, try to split along Markdown headings (starting with level 2)
                "\n#{1,6} ",
                # Note the alternative syntax for headings (below) is not handled here
                # Heading level 2
                # ---------------
                # End of code block
                "```\n",
                # Horizontal lines
                "\n\\*\\*\\*+\n",
                "\n---+\n",
                "\n___+\n",
                # Note that this splitter doesn't handle horizontal lines defined
                # by *three or more* of ***, ---, or ___, but this is not handled
                "\n\n",
                "\n",
                " ",
                "",
            ]
```

As shown in the simple example:
- The chunk size and chunk overlap define the size of the text chunk. You can play around with these parameters and see how they affect the output.
- No metadata is added or maintained during the splitting process.


In [ ]:
# Document Specific Splitting - Markdown
from langchain.text_splitter import MarkdownTextSplitter
splitter = MarkdownTextSplitter(chunk_size = 50, chunk_overlap=5)
markdown_text = """
# Fun in California

## Driving

Try driving on the 1 down to San Diego

### Food

Make sure to eat a burrito while you're there

## Hiking

Go to Yosemite
"""
print(splitter.create_documents([markdown_text]))

# Langchain's MarkdownHeaderTextSplitter
The `MarkdownHeaderTextSplitter` does not inherit from `RecursiveCharacterTextSplitter`.  It's chunk size is defined by the header level specified in the `headers_to_split_on` list. This could mean really large or small chunks since it is based on the user's choice of headers. I could imagine an approach that starts here and then uses a `RecursiveCharacterTextSplitter` to clean up chunks broken on the header that are too large.

Play around with the `headers_to_split_on` list to see how the splitting behaves.
Notice:
- The headers are included in the text of the chunk as well as the metadata.
- The chunk size is defined by the header level.


In [5]:

markdown_text = """#kis_book_club #teaming_with_microbes,  #rhyzosphere, #mycorrhiza_fungi
# KIS Patreon Book Club 03/19/2024: Chapter 1, "Teaming with Microbes"

## Introduction

The KIS Patreon Book Club's recent session, led by Tad with notable contributions from Brandon, dove into "Teaming with Microbes" by Jeff Lowenfels and Wayne Lewis. This seminal work introduces gardeners to the soil food web's critical role in organic gardening. The book's emphasis on shifting from chemical to microbial gardening practices sparked an enlightening discussion among club members, who shared their journeys towards embracing organic gardening methods.

## The Triangle of Soil Health

Tad introduced a foundational concept, the Triangle of Soil Health, which encapsulates the interdependence of soil's physical, chemical, and biological properties. This model underpins the book's approach, advocating for a balanced focus on enhancing soil structure, nutrient content, and microbial diversity to foster a healthy garden ecosystem.

### Key Insights

Physical Properties: Soil's texture and structure create a conducive environment for microorganisms and plant roots.
Chemical Properties: Soil chemistry, including nutrient content and pH levels, influences microbial activity and plant health.
Biological Properties: The diverse microbial ecosystem within the soil supports nutrient cycling and disease suppression.
The Impact of Chemical Fertilizers
The discussion highlighted concerns regarding chemical fertilizers' potential to disrupt soil microbiology. Tad referenced Dr. Elaine Ingham's analogy comparing chemical fertilizers to "candy" for microbes—a short-term energy source lacking nutritional value. The conversation underscored the need for more research to fully understand these impacts and emphasized organic practices as a preferable alternative.

# Mycorrhizal Fungi: Symbiotic Partners

Mycorrhizal fungi's role as symbiotic partners to plants was a focal point. These fungi enhance nutrient and water uptake, contributing to plant health and soil structure. The club discussed practices to promote mycorrhizal fungi, such as minimizing soil disturbance and avoiding chemical inputs.

## The Rhizosphere

The rhizosphere is the soil zone immediately surrounding plant roots. It extends just a few millimeters from the root surface, yet it's a hotbed of biological activity where roots and soil life interact closely. One of the most fascinating revelations about the rhizosphere is that plants allocate a substantial portion of their energy—30% to 40% of the carbon fixed during photosynthesis—towards creating root exudates. These carbon-rich compounds, including sugars and amino acids, are secreted through the root hairs into the rhizosphere, attracting a diverse microbial population.

This strategic release of nutrients by the plants not only supports a thriving microbial ecosystem but also plays a crucial role in facilitating nutrient exchange and protection against pathogens. The discussion emphasizes how this significant energy investment into the rhizosphere exemplifies the plant's active role in engineering its immediate environment, tailoring the microbial community to its current needs. This insight into the rhizosphere's dynamics showcases the intricate and symbiotic relationships between plants and microbes, underscoring the importance of maintaining the health of this critical zone through practices that promote microbial diversity and activity.

## Closing
It's clear this book serves as a gateway to many mind-blowing observations about the soil's microbial community. With revelations such as a single teaspoon of good garden soil hosting over a billion invisible bacteria and containing thousands of different species, the book underscores the astonishing complexity and diversity hidden beneath our feet. While "Teaming with Microbes" implies numerous positive outcomes of nurturing this microscopic world—ranging from enhanced plant health to more resilient ecosystems—the specifics of these benefits, though suggested, are often left for the reader to witness firsthand in their gardening practices. This gap between the implications and explicit discussions invites gardeners to embark on their own journey of discovery, emphasizing that each scoop of soil is not just earth but a vibrant, teeming universe awaiting our understanding and appreciation."""


In [ ]:
from rich import print
from langchain.text_splitter import MarkdownHeaderTextSplitter
headers_to_split_on = [
    ("#", "Header 1"),
    # ("##", "Header 2"),
    # ("###", "Header 3"),
]
splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on,strip_headers=False)

print(splitter.split_text(markdown_text))

# Split Obsidian Notes
Based on the above, I'm focusing on the `MarkdownHeaderTextSplitter` for now. Let's large document and evaluate how text splitting looks.

In [ ]:
from src.ingest_service import IngestService
from src.doc_stats import DocStats
ingest_service = IngestService()
obsidian_notes_path = 'eval/obsidian_notes'
# obsidian_notes_path = r'G:\My Drive\Audios_To_Knowledge\knowledge\AskGrowBuddy\AskGrowBuddy\Knowledge\soil_test_knowlege'
docs = ingest_service.load_obsidian_notes(obsidian_notes_path)

DocStats.print_llama_index_docs_summary_stats(docs)


In [ ]:
from src.ingest_service import IngestService
ingest_service = IngestService()
nodes = ingest_service.chunk_text(docs)


DocStats.print_llama_index_docs_summary_stats(nodes)


In [ ]:
from node_view import launch_node_viewer
launch_node_viewer(nodes)